## <b><font color='darkblue'>Preface</font></b>
([course link](https://learn.deeplearning.ai/courses/functions-tools-agents-langchain/lesson/6/tools-and-routing)) <b><font size='3ptx'>Tools are interfaces that an agent, chain, or LLM can use to interact with the world.</font></b>

They combine a few things ([more](https://python.langchain.com/v0.1/docs/modules/tools/)):
* The name of the tool
* A description of what the tool is
* JSON schema of what the inputs to the tool are
* The function to call
* Whether the result of a tool should be returned directly to the user

![ch6_1](images/ch6_tooling_and_routing_1.png)

In [1]:
!pip freeze | grep -P '(openai|langchain)'

langchain==0.2.6
langchain-anthropic==0.1.15
langchain-community==0.2.6
langchain-core==0.2.10
langchain-experimental==0.0.62
langchain-google-genai==1.0.6
langchain-groq==0.1.3
langchain-openai==0.1.9
langchain-text-splitters==0.2.0
langchainhub==0.1.14
openai==1.28.1


In [68]:
import json
import os
import openai
import re
import httpx
import os
from dotenv import load_dotenv, find_dotenv
from typing import List, Optional
from pydantic import BaseModel, Field, TypeAdapter

import openai
from openai import OpenAI
from langchain.agents import tool
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.tools.render import format_tool_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## <b><font color='darkblue'>Agent Tool Anatomy</font></b>
<b><font size='3ptx'>When constructing an agent, you will need to provide it with a list of `Tool`s that it can use. ([More on how to create tools](https://python.langchain.com/v0.2/docs/how_to/custom_tools/))</font></b>

Besides the actual function that is called, the `Tool` consists of several components:
* **name** (str): Must be unique within a set of tools provided to an LLM or agent.
* **description** (str): Describes what the tool does. Used as context by the LLM or agent.
* **args_schema** (Pydantic BaseModel): Optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters
* **return_direct** (bool): Only relevant for agents. When True, after invoking the given tool, the agent will stop and return the result direcly to the user.

Creating tools from functions may be sufficient for most use cases, and can be done via a simple [@tool](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.tool.html#langchain_core.tools.tool) decorator. 

### <b><font color='darkgreen'>Creating tools from functions</font></b>
<b><font size='3ptx'>This [**@tool**](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.tool.html#langchain_core.tools.tool) decorator is the simplest way to define a custom tool. ([more](https://python.langchain.com/v0.2/docs/how_to/custom_tools/#creating-tools-from-functions))</font></b>

The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided.

In [4]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [5]:
search.name

'search'

In [6]:
search.description

'Search for weather online'

In [7]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [11]:
from langchain.pydantic_v1 import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [12]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [13]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [14]:
search.run("sf")

'42f'

### <b><font color='darkgreen'>Tool to get currrent temperature</font></b>

In [16]:
import requests
from langchain.pydantic_v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [17]:
get_current_temperature.name

'get_current_temperature'

In [18]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [19]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [23]:
convert_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [25]:
get_current_temperature.invoke({"latitude": 13, "longitude": 14})

'The current temperature is 24.9°C'

### <b><font color='darkgreen'>Tool to query Wikipedia</font></b>

In [26]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [27]:
search_wikipedia.name

'search_wikipedia'

In [28]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [29]:
convert_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [30]:
search_wikipedia.invoke({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a type of information retrieval process. It modifies interactions with a large language model (LL

### <b><font color='darkgreen'>OpenAPI Spec</font></b>
<b><font size='3ptx'>The OpenAPI Specification (OAS) defines a standard, language-agnostic interface to HTTP APIs which allows both humans and computers to discover and understand the capabilities of the service without access to source code, documentation, or through network traffic inspection</font></b>.

When properly defined, a consumer can understand and interact with the remote service with a minimal amount of implementation logic. ([more](https://swagger.io/specification/))

In [46]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec
from openapi_pydantic import OpenAPI, Info, PathItem, Operation, Response

In [47]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [49]:
# Need to resolve
# AttributeError: 'super' object has no attribute 'parse_obj'
# spec = OpenAPISpec.from_text(text)

### <b><font color='darkgreen'>Routing</font></b>
<b><font size='3ptx'>In lesson 3, we show an example of function calling deciding between two candidate functions.</font></b>

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [51]:
functions = [
    convert_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [52]:
model.invoke("What is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 105, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f0ec7948-17a4-4d9d-82ad-879ca917f29b-0', usage_metadata={'input_tokens': 105, 'output_tokens': 25, 'total_tokens': 130})

In [53]:
model.invoke("What is langchain?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 102, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b11e5693-5adf-4bfa-9dc2-11513a229e9b-0', usage_metadata={'input_tokens': 102, 'output_tokens': 16, 'total_tokens': 118})

In [54]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [56]:
chain.invoke({"input": "What is the weather in Taipei right now?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":25.033,"longitude":121.5654}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 114, 'total_tokens': 138}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-4e6926dc-e394-47b6-8c76-0226ea6c2257-0', usage_metadata={'input_tokens': 114, 'output_tokens': 24, 'total_tokens': 138})

In [58]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [59]:
result = chain.invoke({"input": "What is the weather in Taipei right now?"})

In [60]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [61]:
result.tool

'get_current_temperature'

In [62]:
result.tool_input

{'latitude': 25.033, 'longitude': 121.5654}

In [63]:
get_current_temperature.invoke(result.tool_input)

'The current temperature is 34.5°C'

In [64]:
result = chain.invoke({"input": "Hi!"})

In [65]:
type(result)

langchain_core.agents.AgentFinish

In [66]:
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

In [69]:
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [70]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [71]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [72]:
result

'The current temperature is 14.6°C'

In [73]:
result = chain.invoke({"input": "What is langchain?"})

In [74]:
result

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a type of information retrieval process. It modifies interactions with a large language model (LL

In [75]:
chain.invoke({"input": "Hi!"})

'Well, hello there! How can I assist you today?'